In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

# 6.3. Optimización de carteras

- Descarga los datos del último año de los siguientes activos usando IEX ('AAPL', 'FB', 'JPM', 'BRK.B', 'GOOG'). Denominamos $n$ como el número de activos elegidos.
- Crea un dataframe solo con los datos de cierre, tendrá dimensiones $m \times n$ con $m$ el número de días y $n$ en número de activos.
- Calcula un portfolio aleatorio:
   - Genera uno vector de pesos $\mathbf{w}$ aleatorios de dimensiones $n \times 1$, donde $n$ es el número de activos. Los pesos tienen que cumplir que  $w_i \ge 0, \; i = 1,\dots, n$ y $\sum_{i=1}^{n} w_i = 1$ dado que solo permitimos estar en largo.
   - Calcula el retorno y la desviacion típica del portfolio:
       - $R_p = \mathbf{w^T \mu}$ donde $\mathbf{\mu}$ es un vector con la media de los retornos diarios de los activos, con dimensiones $n \times 1$.
       - $\sigma_p = \sqrt{\mathbf{w}^T\Sigma \mathbf{w}}$, donde $\Sigma$ es la matriz de covarianzas de los retornos, con dimensiones $n \times n$
- Realiza una simulación de montecarlo para ello:
   - Repite el procceso anterior $k$ veces por ejemplo 10000, guardando los pesos $\mathbf{w}$, $R_p$ y $\sigma_p$ de cada experimento.
   - Realiza una figura con todos los puntos.
   - Selecciona el punto de máximo ratio sharpe:
     - Calcula el ratio sharpe para cada uno de los porfolios generados. El ratio sharpe se calcula como:
     \begin{equation*}
     S_p = \frac{R_p - R_f}{\sigma_p}
     \end{equation*}
     asumimos que $R_f$, el retorno del activo sin riesgo, es 0. 
     - Obtén el porfolio con el $S_p$ mayor.
     - Pinta este punto en la figura y obtén los pesos.
      
<center>
<img src="imgs/harry.jpeg"  alt="drawing" width="300"/>
<br />    Harry Markowitz, Novel 1990
</center>

  
   

### Descarga de datos

- Descarga los datos de cierre, del último año, de los siguientes tickers: 'AAPL', 'FB', 'JPM', 'BRK.B', 'GOOG'

- Unimos los datos de cierre de los 5 activos en un único DataFrame

- Graficalo, para ver qué pinta tiene la evolución de las acciones. Tienes que hacer es dividir cada acción por la 1ª cotización (para que todas empiecen en el mismo punto).

### Random porfolio

- Vamos a generar 10.000 vectores de pesos aleatorios w, que sumen 1.
- Para cada vector de pesos, vamos a calcular la rentabilidad de la cartera y el riesgo de la cartera
    - La rentabilidad de la cartera se calcula multiplicando, matricialmente, el vector de media de los retornos * el vector de pesos
    - El riesgo de la cartera se calcula como la raiz cuadrada de: la multiplicación matricial del vector de pesos por la matriz de covarianzas, multiplicada de nuevo por el vector de pesos (matricialmente).

Dibujamos todas las carteras que hemos generado. 

- Eje X: Riesgo
- Eje y: Rentabilidad

#### Max sharpe ratio portfolio

- Calculamos la eficiencia de la cartera a través del ratio de Sharpe: rentabilidad / riesgo
- Calculamos el ratio de Sharpe para cada cartera.
- La cartera que tenga mayor ratio de Sharpe, será la más eficiente.

- Recuperamos la información la cartera óptima encontrada: rentabilidad, riesgo y pesos

#### Plot max sharpe ratio portfolio

Dibulamos todas las carteras, y marcamos la cartera óptima encontrada

## Math opt con cvxpy

El problema de la frontera eficiente se puede tratar como un problema de optimización cuadrática.

La frontera eficiente se puede encontrar solucionando el siguiente problema:

\begin{equation*}
\begin{aligned}
& \underset{}{\text{maximize}}
& &  \mathbf{w^T \mu} - \gamma \mathbf{w}^T\Sigma \mathbf{w}\\
& \text{subject to}
& & w_i \ge 0, \; i = 1,\dots, n \\
&&& \sum_{i=1}^{n} w_i = 1
\end{aligned}
\end{equation*}

Donde $\mathbf{w}$ es el vector de pesos a optimizar, $\mathbf{w^T \mu}$ es el retorno del portfolio, $\mathbf{w}^T\Sigma \mathbf{w}$ es la varianza del portfolio y $\gamma$ es un parámetro de adversión al riesgo.

Variando $\gamma$ obtenemos los diferentes puntos de la frontera. 
- Si $\gamma=0$ maximizamos solo el retono 
- Si  $\gamma \rightarrow \infty$ obtenemos el portfolio de mínimo riesgo. 

Variando  $\gamma$  obtenemos el *risk-return trade-off* óptimo.

Puedes encontrar más información en: https://en.wikipedia.org/wiki/Modern_portfolio_theory

Para ejecutar el siguiente código necesitas installar cvxpy: ```pip install cvxpy```

In [ ]:
import cvxpy as cp

In [ ]:
def optimal_portfolio(returns, n_samples=1000, gamma_low=-1, gamma_high=10):
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T*w
    risk = cp.quad_form(w, sigma)
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
                      [cp.sum(w) == 1,  w >= 0]) 
    
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []    
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve()
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)   
    return ret_data, risk_data, gamma_vals, portfolio_weights

In [ ]:
ret_data, risk_data, gamma_vals, portfolio_weights = optimal_portfolio(returns)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
plt.plot(list_std_p, list_r_p, '.')
ax.plot(risk_data, ret_data, 'y')
ax.plot(std_m_sr, r_m_sr, '*', markersize=15)
_ = ax.set_xlabel('std')
_ = ax.set_ylabel('mean')

# Conclusiones
- ¿Os dáis cuenta de que la frontera está muy por encima de las simulaciones que hemos realizado?, ¿sabríais decirme el motivo?
- Estamos trabajando con una foto: ¿Qué significa esta frase?
- La optimización de Markowitz tiene diversas críticas. ¿Sabríais decirme alguna?